In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

%%time
from pandas import DataFrame

from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

from hc_utils import HeaderCategories
hc = HeaderCategories(cu=cu, verbose=False)

from section_utils import SectionUtilities
su = SectionUtilities(s=s, ha=ha, cu=cu, verbose=False)

from lr_utils import LrUtilities
lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
lru.build_isheader_logistic_regression_elements()
lru.build_pos_logistic_regression_elements()

import warnings
warnings.filterwarnings('ignore')

CPU times: total: 35.8 s
Wall time: 41.4 s


In [4]:

%run ../load_magic/dataframes.py
from datetime import datetime
from urllib.parse import urlparse, parse_qs


----

In [24]:

from urllib.error import HTTPError

file_path = '../data/html/indeed_email.html'
page_soup = get_page_soup(file_path)
css_selector = 'body > table > tbody > tr > td > a > table > tbody > tr > td > a'
link_soups_list = page_soup.select(css_selector)
files_list = []
for link_soup in link_soups_list:
    url_str = link_soup['href']
    try:
        page_soup = get_page_soup(url_str)
        page_title = page_soup.find('title').string.strip()
        file_name = re.sub(r'\W+', ' ', page_title).strip().replace(' ', '_')
        jk = parse_qs(urlparse(url_str).query).get('jk', [''])[0]
        if len(jk):
            file_name = f'{jk}_{file_name}.html'
        else:
            # file_name = datetime.now().strftime('%Y%m%d%H%M%S%f') + f'_{file_name}.html'
            file_name = f'{file_name}.html'
        file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
        if not os.path.isfile(file_path):
            with open(file_path, 'w', encoding=s.encoding_type) as f:
                print(f'Saving to {file_path}')
                f.write('<html><head><title>')
                f.write(page_title)
                f.write('</title></head><body>')
                row_div_list = page_soup.find_all(name='div', attrs={'class': ['jobsearch-JobComponent-description']})
                for div_soup in row_div_list:
                    f.write(str(div_soup))
                f.write('</body></html>')
            files_list.append(file_name)
        cu.ensure_filename(file_name, verbose=False)
        file_name = cu.clean_text(file_name)
        url_str = cu.clean_text(url_str)
        cypher_str = f'''
            MATCH (fn:FileNames {{file_name: "{file_name}"}})
            SET fn.posting_url = "{url_str}"
            RETURN fn;'''
        with cu.driver.session() as session:
            row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
            # print(row_objs_list)
    except HTTPError as e:
        print(f'Got an HTTPError with {url_str}: {str(e).strip()}')
    except Exception as e:
        print(f'Got an {e.__class__} error with {url_str}: {str(e).strip()}')

Got an HTTPError with https://www.indeed.com/rc/clk/dl?jk=e6ea2963f2bf7038&from=ja&qd=RnZhMybXSk4M3QtTVGXWoZj-R_bxcYib5xeGNtZ7GZZCz9IDoAfw9Pn-C-g9mkpdHI4neQgiQceQG2P-Q_YyG8TLUsj70s3Sqe1WWe88NIQ&rd=AbLHrm6GRQMtYBMzyEs2L1_MKnaSAFGAsD6kfERFt3g&tk=1g5fg2ha2g78g800&alid=6254377b33b425113af40aa2: HTTP Error 404:
Got an HTTPError with https://www.indeed.com/rc/clk/dl?jk=c29833f9f645ad3e&from=ja&qd=RnZhMybXSk4M3QtTVGXWoZj-R_bxcYib5xeGNtZ7GZZCz9IDoAfw9Pn-C-g9mkpdHI4neQgiQceQG2P-Q_YyG8TLUsj70s3Sqe1WWe88NIQ&rd=F-EaRGHMzpZKxB2tk3D1Ml_MKnaSAFGAsD6kfERFt3g&tk=1g5fg2ha2g78g800&alid=6254377b33b425113af40aa2: HTTP Error 404:


In [23]:

verbose = True
cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    for div_soup in row_div_list:
        child_strs_list = ha.get_navigable_children(div_soup, [])
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)